In [1]:
!pip install implicit
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 34.2 MB/s eta 0:00:00


In [11]:
import os
import time
import numpy as np
import pandas as pd

import faiss


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import os
os.getcwd()

'/content'

## Load Data

In [8]:
def ivecs_read(fname):
  a = np.fromfile(fname, dtype='int32')
  d = a[0]
  return a.reshape(-1, d + 1)[:, 1:].copy()

In [20]:
# download data from http://corpus-texmex.irisa.fr/ 
xq = ivecs_read('/content/drive/MyDrive/movie_lens/ml-100k/sift/sift_query.fvecs').view('float32')
xb = ivecs_read('/content/drive/MyDrive/movie_lens/ml-100k/sift/sift_base.fvecs').view('float32')
gt = ivecs_read('/content/drive/MyDrive/movie_lens/ml-100k/sift/sift_groundtruth.ivecs').view('float32')
xt = ivecs_read('/content/drive/MyDrive/movie_lens/ml-100k/sift/sift_learn.fvecs').view('float32')

In [21]:
nq, d = xq.shape

In [22]:
faiss.omp_set_num_threads(20)

### HNSW Flat Index

In [ ]:
index = faiss.IndexHNSWFlat(d, 32)
index.hnsw.efConstruction = 40
index.verbose = True
index.add(xb)

In [ ]:
for efSearch in [16, 32, 64, 128, 256]:

  index.hnsw.efSearch = efSearch

  t0 = time.time()
  D, I = index.search(xq, 100)
  t1 = time.time()

  missing_rate = (I == -1).sum() / float(100 * nq)
  recall_at_1 = (I == gt[:, :1]).sum() / float(nq)
  print("%7.3f ms per query, R@1 %.4f" %(
        (t1 - t0) * 1000.0 / nq, recall_at_1))

  0.158 ms per query, R@1 0.9123
  0.240 ms per query, R@1 0.9614
  0.393 ms per query, R@1 0.9872
  0.827 ms per query, R@1 0.9959
  1.411 ms per query, R@1 0.9987


### HNSW Hyperparameter Test

In [ ]:
hnsw_results = []

m_vals_list = [8, 12, 16, 24, 32, 64]
efconstruction_vals_list = [4, 8, 12, 16, 24, 32]
efsearch_vals_list = [4, 8, 12, 16, 24, 32]


for M in m_vals_list:
  for efConstruction in efconstruction_vals_list:
    index = faiss.IndexHNSWFlat(d, M)
    index.hnsw.efConstruction = efConstruction
    index.verbose = True

    T0 = time.time()
    index.add(xb)
    T1 = time.time()

    print('Time taken for construction of HNSW index with M = {0} and efConstruciton = {1} is {2:.3f} s'.format(M, efConstruction, (T1-T0)))

    for efSearch in efsearch_vals_list:
      index.hnsw.efSearch = efSearch

      t0 = time.time()
      D, I = index.search(xq, 100)
      t1 = time.time()

      recall_at_1 = (I == gt[:, :1]).sum() / float(nq)
      print("For efSearch = %5d, Search Time = %7.3f ms per query, Recall@1 = %.4f" %(efSearch,
            (t1 - t0) * 1000.0 / nq, recall_at_1))

      hnsw_results.append(pd.DataFrame({'m' : [M], 'efSearch' : [efSearch], 'efConstruction' : [efConstruction], 'Search_Time_Per_Query' : [(t1 - t0) * 1000.0 / nq], 'Recall@1' : [recall_at_1]}))

    print('*'*100)

Time taken for construction of HNSW index with M = 8 and efConstruciton = 4 is 25.596 s
For efSearch =     4, Search Time =   0.022 ms per query, Recall@1 = 0.1334
****************************************************************************************************
For efSearch =     8, Search Time =   0.030 ms per query, Recall@1 = 0.1593
****************************************************************************************************
For efSearch =    12, Search Time =   0.040 ms per query, Recall@1 = 0.1797
****************************************************************************************************
For efSearch =    16, Search Time =   0.048 ms per query, Recall@1 = 0.1967
****************************************************************************************************
For efSearch =    24, Search Time =   0.069 ms per query, Recall@1 = 0.2185
****************************************************************************************************
For efSearch =    32, Searc

In [ ]:
hnsw_results_df = pd.concat(hnsw_results, axis = 0)
hnsw_results_df

,m,efSearch,efConstruction,Search_Time_Per_Query,Recall@1
0,8,4,4,0.022081,0.1334
0,8,8,4,0.029712,0.1593
0,8,12,4,0.040308,0.1797
0,8,16,4,0.047703,0.1967
0,8,24,4,0.068829,0.2185
...,...,...,...,...,...
0,64,8,32,0.177414,0.8558
0,64,12,32,0.218144,0.9056
0,64,16,32,0.396230,0.9309
0,64,24,32,0.364841,0.9599


In [ ]:
!mkdir -p "/content/drive/My Drive/Temp Results"

In [ ]:
hnsw_results_df.to_csv('/content/drive/My Drive/Temp Results/hnsw_sift1m_results.csv')

In [ ]:
exact_index = faiss.index_factory(d, "Flat")
exact_index.add(xb)

In [ ]:
t0 = time.time()
D, I = exact_index.search(xq, 100)
t1 = time.time()

recall_at_1 = (I == gt[:, :1]).sum() / float(nq)
print("Search Time = %7.3f ms per query, Recall@1 = %.4f" %(
            (t1 - t0) * 1000.0 / nq, recall_at_1))

Search Time =   7.965 ms per query, Recall@1 = 1.0000


### HNSW with Quantizer

In [ ]:
index = faiss.IndexHNSWSQ(d, faiss.ScalarQuantizer.QT_8bit, 16)

In [ ]:
index.train(xt)
index.hnsw.efConstruction = 40
index.hnsw.efSearch = 32

index.verbose = True
index.add(xb)

In [ ]:
for efSearch in 16, 32, 64, 128, 256:

  index.hnsw.efSearch = efSearch
  t0 = time.time()
  D, I = index.search(xq, 100)
  t1 = time.time()

  missing_rate = (I == -1).sum() / float(100 * nq)
  recall_at_1 = (I == gt[:, :1]).sum() / float(nq)
  print("%7.3f ms per query, R@1 %.4f" %(
          (t1 - t0) * 1000.0 / nq, recall_at_1))

  0.160 ms per query, R@1 0.8026
  0.241 ms per query, R@1 0.8934
  0.224 ms per query, R@1 0.9492
  0.346 ms per query, R@1 0.9796
  0.782 ms per query, R@1 0.9921


### IVF with HNSW Quantizer

In [ ]:
quantizer = faiss.IndexHNSWFlat(d, 32)
index = faiss.IndexIVFFlat(quantizer, d, 16384)
index.cp.min_points_per_centroid = 5
index.quantizer_trains_alone = 2

In [ ]:
index.verbose = True
index.train(xt)
index.add(xb)

In [ ]:
quantizer.hnsw.efSearch = 64
for nprobe in 1, 4, 16, 64, 256:
  index.nprobe = nprobe

  t0 = time.time()
  D, I = index.search(xq, 100)
  t1 = time.time()

  missing_rate = (I == -1).sum() / float(100 * nq)
  recall_at_1 = (I == gt[:, :1]).sum() / float(nq)
  print("%7.3f ms per query, R@1 %.4f" %(
          (t1 - t0) * 1000.0 / nq, recall_at_1))

  0.088 ms per query, R@1 0.4094
  0.109 ms per query, R@1 0.6323
  0.177 ms per query, R@1 0.8299
  0.395 ms per query, R@1 0.9549
  1.390 ms per query, R@1 0.9927
